In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM, PeftModel
from cut_cross_entropy.transformers import cce_patch
from cut_cross_entropy import linear_cross_entropy
import torch
import transformers
import numpy as np
import random
from torchviz import make_dot

In [3]:
tokenizer = AutoTokenizer.from_pretrained('HuggingFaceTB/SmolLM2-135M-Instruct')

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    'HuggingFaceTB/SmolLM2-135M-Instruct',
    torch_dtype = torch.bfloat16
).cuda()

In [ ]:
rank = 64
peft_config = LoraConfig(
        lora_alpha=rank * 2,
        lora_dropout=0.0,
        r=rank,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["embed_tokens", "lm_head"],
    )

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
input_ids = torch.tensor([1,2,3])[None].cuda()

In [ ]:
o = model(input_ids = input_ids, output_hidden_states = True)

In [ ]:
classifier = model.lm_head.weight

In [ ]:
classifier.shape

In [ ]:
c_a = model.lm_head.lora_A.default.weight
c_b = model.lm_head.lora_B.default.weight
alpha = model.lm_head.scaling['default']

In [ ]:
manual_shift_loss = linear_cross_entropy(o.hidden_states[-1], classifier,
                                         c_a = c_a,
                                         c_b = c_b,
                                         alpha = alpha,
                                         targets = input_ids, shift = True)

In [21]:
manual_shift_loss

tensor(19.7411, device='cuda:0', grad_fn=<LinearCrossEntropyFunctionBackward>)

In [18]:
l = model(input_ids = input_ids, labels = input_ids).loss

In [19]:
l.backward()

In [22]:
manual_shift_loss.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [23]:
c_b.grad

tensor([[-2.9976e-04, -1.3902e-04, -2.0747e-06,  ..., -7.0202e-05,
         -6.8997e-05,  3.2948e-04],
        [-2.6686e-08, -1.4672e-08,  5.7066e-10,  ...,  2.7285e-10,
         -5.9582e-09,  2.4281e-08],
        [ 1.6168e+00,  7.2701e-01, -1.1361e-02,  ...,  3.5186e-01,
          5.2212e-01, -1.6118e+00],
        ...,
        [-5.8421e-07, -6.5146e-07, -2.1739e-07,  ..., -9.5092e-08,
          2.2800e-07,  9.7687e-07],
        [-7.0167e-06, -6.6236e-06, -1.7752e-06,  ..., -7.7464e-07,
          1.4352e-06,  1.0114e-05],
        [-4.4343e-09, -4.9568e-09, -1.6584e-09,  ..., -7.2544e-10,
          1.7436e-09,  7.4309e-09]], device='cuda:0')

In [17]:
tensor([[ 4.8602e-05,  9.1227e-05, -2.9040e-05,  ...,  1.8487e-05,
         -3.2635e-05,  8.4391e-06],
        [ 6.7039e-09,  1.0294e-08, -2.8363e-09,  ...,  1.3824e-09,
         -2.6497e-09,  1.2757e-09],
        [-3.8740e-01, -5.9367e-01,  1.6330e-01,  ..., -7.9289e-02,
          1.5217e-01, -7.3774e-02],
        ...,
        [-1.8596e-07, -3.0567e-08, -5.1531e-08,  ...,  9.1656e-08,
         -1.3270e-07, -4.7815e-08],
        [-1.0237e-06,  5.6337e-07, -6.5732e-07,  ...,  8.7762e-07,
         -1.3222e-06, -2.9889e-07],
        [-1.4235e-09, -2.4130e-10, -3.9075e-10,  ...,  6.9792e-10,
         -1.0100e-09, -3.6568e-10]], device='cuda:0')

NameError: name 'tensor' is not defined